<a href="https://colab.research.google.com/github/Thanawich-Pinprayoon/Dog-and-Cat-Lab-test-to-medicine-recommendation/blob/master/make_all_model_and_stat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Predict_use_this_medicine?_decisionTree.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1PRIrtT8MD_qcHVp_Bmq61zy7Dgx3gXbJ
"""
"""#import lib"""

import os 
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import pickle
from sklearn.tree import export_graphviz
import graphviz



"""#import file"""

traininput = pd.read_csv('/content/drive/My Drive/ThaiWordcutTest/Input CSV/11month_data_in.csv')

trainAnswerFile = pd.read_csv('/content/drive/My Drive/ThaiWordcutTest/OutputCSV/GP/trainAnswer_GP_12month.csv')

"""#get medicine list"""

med_list = []
for index,_med in trainAnswerFile.iterrows():
  medd = str(_med['Name'])
  if medd not in med_list:
    med_list.append(medd)
med_list.sort()
med_list

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


['(ถ้าอาการไม่รุนแรงมาก) Azithromycin ',
 'Amikacin ',
 'Amoxicillin ',
 'Amoxicillin/clavulanic acid',
 'Amoxicillin/clavulanic acid ',
 'Amoxicillin/clavulanic acid Amoxicillin/clavulanic acid ',
 'Azithromycin ',
 'Cefazolin ',
 'Cefixime ',
 'Cefovecin ',
 'Ceftriaxone ',
 'Cephalexin',
 'Cephalexin ',
 'Clindamycin ',
 'Doxycycline ',
 'Enrofloxacin ',
 'Gentamicin ',
 'Marbofloxacin ',
 'Nitrofurantoin ',
 'Pradofloxacin ',
 'Rifampicin ',
 'Rifampicin (rifampin) ',
 'Sulfamethoxazole/trimethoprim ',
 'Vancomycin',
 'Vancomycin ',
 'amikacin',
 'amikacin ',
 'amikacin  ',
 'amikacin หรือ enrofloxacin (ตามขนาดข้างล่าง) หากอาการรุนแรงมาก Amikacin ',
 'antiseptics เป็นส่วนประกอบที่มีคุณสมบัติในการทำลายเชื้อและ biofilm ของเชื้อ หรือ Tris EDTA ทิ้งไว้ ',
 'enrofloxacin ',
 'enrofloxacin  ',
 'enrofloxacin (ตามขนาดข้างล่าง) หากอาการรุนแรงมาก Enrofloxacin ',
 'marbofloxacin ',
 'nan']

In [ ]:
med_list = ['Amikacin',
 'Amoxicillin',
 'Clavulanic acid',
 'Azithromycin',
 'Cefazolin',
 'Cefixime',
 'Cefovecin',
 'Ceftriaxone',
 'Cephalexin',
 'Cephalexin',
 'Clindamycin',
 'Doxycycline',
 'Enrofloxacin',
 'Gentamicin',
 'Marbofloxacin',
 'Nitrofurantoin',
 'Pradofloxacin',
 'Rifampicin',
 'Sulfamethoxazole',
 'Vancomycin',
 'Enrofloxacin',
 'Marbofloxacin']

In [ ]:
stat_df = pd.DataFrame(columns=['medicine','sample amount','score','mean','std'])

In [ ]:
stat_df

,medicine,sample amount,score,mean,std


In [ ]:
  traininput = traininput.drop(columns=['collect_method'])
  traininput = traininput.drop(columns=['vitek_id'])
  traininput = traininput.drop(columns=['id'])

In [ ]:
for aMed in med_list:
  medicine = aMed #upper case (use to name the model)
  medicine2 = aMed.lower()

  """# **# get Answer (True/False) from answer file**"""

  trainAnswer = []
  med_volume = []
  id_before = 	201801000	
  id_current = 	201801000	
  count = 0
  answer = False
  volume = "-"

  for index,case in trainAnswerFile.iterrows():
    id_before = id_current
    id_current = case['Case_id']
    if id_before != id_current:
      #print(id_before,' != ', id_current)
      trainAnswer.append(answer)
      if volume not in med_volume and volume != "-": #uniqe volume
        med_volume.append(volume)
      answer = False
      volume = "-"
    case_medicine = case['Name']
    # print(case_medicine)
    if medicine in str(case_medicine): #change nan(float) to str
      count += 1
      answer = True
      volume = case['Volume']
      # print(case_medicine)
    elif medicine2 in str(case_medicine): #change nan(float) to str
      count += 1
      answer = True
      volume = case['Volume']
      # print(case_medicine)
    # else:
    #   print(case_medicine)
  trainAnswer.append(answer)
  if volume not in med_volume and volume != "-": #uniqe volume
    med_volume.append(volume)


  #count

  """# **change samples to 11 type of sample**
  ![alt text](https://lh3.googleusercontent.com/MWMp1KXE5yMjZEptCrUCs2n0DieunVP87sjkLA9_V7gFyQYnhcBoe8OwBa0n1lXMHFxwvSsyQE9krm9PORb5KcAysvQiQ2OByfe1PAcy92N2yAwe4AA64PJGuN5RS1zaniCy-nBKJIGUwclv6uARQ-aA-tkyGSk16eKCKwNzjNmIlGyBcjRHMICfFQ3v-nnmjC6rVJy59BMB81ApctR83JO1lcH43Hj3xDxZ2PfvXACxulZmW192ph4vhrVMZI3i7Zzhv4n3qj2lWIhudq4E6mAasRUMQrAO0rcgxoJuEZnuRzjRo4ydMb5OCMhFP3Pk-WRc_MkeTx6mH4tpmbz-4wANHAKEtT0wz6hIinHDaLzxyGtATBcDUsoh1qgyiRNlAC2QXbry9562sRXRB7sPA8KbP-dIzPl-XJKyll_ZVsyUvsckLstYS4XR8JTIHP8MfN6nRvNhst050A35Dr4BW5YQwDi2FCtVCysiUaZLnGStaUYW63bekGK7OqZUKSfoDbhy5wEgwQvzIWCwy5gBiv7IGPadrVIaLHGhWStpOA2p0EToO19qQUNCBFJ8FY4Sh2j92d7vxcX-DI4A2VfEKITI5rupTdtNIvBarbdT6A27X1lV5TG2o3a5k5wtCJuZi9TBqxtiGrwGnntLXCkHCa3AWmCFKVfdlkQBQgKs5Bj7_ALhnziQfqEpbqtrv8vuOjY3fg=w1057-h937-ft)
  """

  sample_Collect = []
  for index, row in  traininput.iterrows():
    case = row['submitted_sample']
    if case == "wound":
      AppenData = "Wound"
    elif case == "Granulomatous formation @ Lt Stifle (Sx site)":
      AppenData = "ETC..."
    elif case == "Right proximal ureter (calculi)": #Not Sure
      AppenData = "Ub mucosa"
    elif case == "Pustule":
      AppenData = "pustule"
    elif case == "Fistular tract":
      AppenData = "ETC..."
    elif case == "Chin Acne":  #Not Sure
      AppenData = "Abscess"
    elif case == "Peritoneal fluid":
      AppenData = "Abdominal effusion"
    elif case == "UB mucosa":
      AppenData = "Ub mucosa"
    elif case == "Lt. ear exudate":
      AppenData = "Ear exudate"
    elif case == "urine":
      AppenData = "Urine"
    elif case == "Wound @ Lt thorax":
      AppenData = "Wound"
    elif case == "Exudate from surgical wound":
      AppenData = "Surgical Site"
    elif case == "Deep pyoderma":
      AppenData = "ETC..."
    elif case == "Gall bladder":
      AppenData = "Ub mucosa"
    elif case == "pustule":
      AppenData = "pustule"
    elif case == "Purulent exudate (pododermatitis)":   #not SURE
      AppenData = "pustule"
    elif case == "GB":  #Not Sure
      AppenData = "Ub mucosa"
    elif case == "Purulent from mass":
      AppenData = "pustule"
    elif case == "Rt stifle":
      AppenData = "ETC..."
    elif case == "Nasal discharge": #not Sure
      AppenData = "Pleural effusion"
    elif case == "Crust from skin":
      AppenData = "ETC..."
    elif case == "Left thoracic wound":
      AppenData = "Wound"
    elif case == "Oral mass":
      AppenData = "ETC..."
    elif case == "Chronic wound":
      AppenData = "Wound"
    elif case == "Exudate from PG cyst": # not Sure
      AppenData = "ETC..."
    elif case == "Wound @ lt lumbar":
      AppenData = "Wound"
    elif case == "Pus from renal pelvis":
      AppenData = "ETC..."
    elif case == "Wound @ Lt flank":
      AppenData = "Wound"
    elif case == "Abdominal effusion":
      AppenData = "Abdominal effusion"
    elif case == "Exudate from wound (bed sore)":
      AppenData = "Wound"
    elif case == "Urine":
      AppenData = "Urine"
    elif case == "Abscess @ right ear":
      AppenData = "Abscess"
    elif case == "Left kidney":
      AppenData = "ETC..."
    elif case == "Epidermal collarette":
      AppenData = "ETC..."
    elif case == "Purulent exudate from wound at Rt.lumbar area":
      AppenData = "Wound"
    elif case == "Wound @ rt perineal area":
      AppenData = "Wound"
    elif case == "แผลโดนเหล็กเสียบ":
      AppenData = "Wound"
    elif case == "Exudate from wound":
      AppenData = "Wound"
    elif case == "Wound at LHL":
      AppenData = "Wound"
    elif case == "Abdominal fluid":
      AppenData = "Abdominal effusion"
    elif case == "Abdo fluid":
      AppenData = "Abdominal effusion"
    elif case == "Bite wound":
      AppenData = "Bite wound"
    elif case == "Crust from skin//Mucocutaneous":
      AppenData = "ETC..."
    elif case == "Exudate from pin protrusion":
      AppenData = "ETC..."
    elif case == "หูขวา":
      AppenData = "Ear exudate"
    elif case == "abscess":
      AppenData = "Abscess"
    elif case == "Rt.stifle hematoma (post MPL surgery)":
      AppenData = "Surgical Site"
    elif case == "furunculosis":
      AppenData = "ETC..."
    elif case == "Deep exudate from celluslitis / deep mycoses / deep pyoderma":
      AppenData = "ETC..."
    elif case == "Pus from rt temporal fistular tract":
      AppenData = "ETC..."
    elif case == "ปัสสาวะ":
      AppenData = "Urine"
    elif case == "Skin swab":
      AppenData = "ETC..."
    elif case == "Purulent exudate from wound":
      AppenData = "Wound"
    elif case == "Exudate from Rt ear":
      AppenData = "Ear exudate"
    elif case == "Exudate from mass":
      AppenData = "ETC..."
    elif case == "Swab from rostral nasal mucosa":
      AppenData = "Pleural effusion"
    elif case == "Skin (post op skin flap)":
      AppenData = "ETC..."
    elif case == "PG abscess":
      AppenData = "Abscess"
    elif case == "Right medial iliac lymph node":
      AppenData = "ETC..."
    elif case == "Open wound at skull (HBC)":
      AppenData = "Wound"
    elif case == "Pleural Effusion":
      AppenData = "Pleural effusion"
    elif case == "Wound (skin laceration)":
      AppenData = "Wound"
    elif case == "Epidermal colleate กิน AMC +Pred จาก clinic Not improve":
      AppenData = "ETC..."
    elif case == "chylothorax":
      AppenData = "ETC..."
    elif case == "Abscess from RT FL":
      AppenData = "Abscess"
    elif case == "Wound @ Rt shoulder":
      AppenData = "Wound"
    elif case == "Wound @ Lt lumbar":
      AppenData = "Wound"
    elif case == "Skin lesion : nodule":
      AppenData = "ETC..."
    elif case == "Lt ear (otitis externa)":
      AppenData = "Ear exudate"
    elif case == "Fluid from retroperitoneal space":
      AppenData = "ETC..."
    elif case == "Wound @ lt neck":
      AppenData = "Wound"
    elif case == "Opened bone fracture":
      AppenData = "ETC..."
    elif case == "Abscess":
      AppenData = "Abscess"
    elif case == "Ub mucosa":
      AppenData = "Ub mucosa"
    elif case == "Nasal pus":
      AppenData = "Pleural effusion"
    elif case == "Ca nasal discharged":
      AppenData = "Pleural effusion"
    elif case == "Pus from pin protusion":
      AppenData = "ETC..."
    elif case == "Pus from abscess":
      AppenData = "Abscess"
    elif case == "Wound @ Lt elbow":
      AppenData = "Wound"
    elif case == "Papulocrustous lesion":
      AppenData = "ETC..."
    elif case == "Chronic open wound":
      AppenData = "Wound"
    elif case == "Pustule not improve after Convenia":
      AppenData = "Abscess"
    elif case == "Surgical site":
      AppenData = "Surgical Site"
    elif case == "Pustule (deep pyoderma)":
      AppenData = "pustule"
    elif case == "Wound exudate":
      AppenData = "Wound"
    elif case == "pus":
      AppenData = "pustule"
    elif case == "Broncho alveolar larvage ในหลอดเพาะเชื้อและในหลอด eppendorf sterile":
      AppenData = "ETC..."
    elif case == "Abdominal cavity":
      AppenData = "Abdominal effusion"
    elif case == "paronychia":
      AppenData = "ETC..."
    elif case == "Bite wound abscess":
      AppenData = "Bite wound"
    elif case == "Infected wound":
      AppenData = "Wound"
    elif case == "Abdo effusion":
      AppenData = "Abdominal effusion"
    elif case == "Rt. tarsal wound":
      AppenData = "Wound"
    elif case == "Exudate from deep pyoderma":
      AppenData = "ETC..."
    elif case == "Open hernia (abdominal cavity)":
      AppenData = "ETC..."
    elif case == "Exudate from mass at dorsal neck":
      AppenData =  "ETC..."
    elif case == "Pericardial fluid":
      AppenData =  "ETC..."
    elif case == "Abscess wound":
      AppenData = "Wound"
    elif case == "Wound @ right axilla area":
      AppenData = "Wound"
    elif case == "Rt nasal disc":
      AppenData = "Pleural effusion"
    elif case == "Rt. thoracic wall (wound abscess)":
      AppenData = "Wound"
    elif case == "Mass at Lt.thorax":
      AppenData =  "ETC..."
    elif case == "Swab แผลที่ขาซ้าย":
      AppenData = "Wound"
    elif case == "Abscess/Exudate":
      AppenData = "Abscess"
    elif case == "หนองจาก cyst ข้างลำตัว":
      AppenData = "Abscess"
    elif case == "Exudate from wound; DDx: Bacterial folliculitis":
      AppenData = "Wound"
    elif case == "Nasal discharge (posterior ventral meatus)":
      AppenData = "Pleural effusion"
    elif case == "Abscess with sinus":
      AppenData = "Abscess"
    elif case == "Wound @ lt pinna":
      AppenData = "Wound"
    elif case == "Pus from wound":
      AppenData = "Wound"
    elif case == "Lt. ear exudate (otitis media)":
      AppenData = "Ear exudate"
    elif case == "Skin (acetate tape : Rod)":
      AppenData =  "ETC..."
    elif case == "Opened wound near dermoid cyst":
      AppenData = "Wound"
    elif case == "Bed sore wounds at hip":
      AppenData = "Wound"
    elif case == "ตุ่มหนองที่ผิวหนังหลังคอ":
      AppenData = "Abscess"
    elif case == "fluid":
      AppenData = "Unknow"
    elif case == "Pulmonary mass":
      AppenData = "ETC..."
    elif case == "Right kidney":
      AppenData = "ETC..."
    elif case == "Abdomen (GI perforate)":
      AppenData = "Abdominal effusion"
    elif case == "Exudate from abscess":
      AppenData = "Abscess"
    elif case == "ก้อนที่ผิวหนังด้านข้างลำตัวฝั่งขวา":
      AppenData = "ETC..."
    elif case == "Pleural effusion":
      AppenData = "Pleural effusion"
    elif case == "Orthopedic operation":
      AppenData = "ETC..."
    elif case == "น้ำดี":
      AppenData = "ETC..."
    elif case == "Rt stifle abscess":
      AppenData = "Abscess"
    elif case == "swab":
      AppenData = "ETC..."
    elif case == "pyoderma":
      AppenData = "ETC..."
    elif case == "Wound exudate แผลกดทับ":
      AppenData = "Wound"
    elif case == "Left TECA":
      AppenData = "ETC..."
    elif case == "Cystic calculi":
      AppenData = "ETC..."
    elif case == "Exudate from marsupial prostatic abscess":
      AppenData = "Abscess"
    elif case == "Mass dorsal back":
      AppenData = "ETC..."
    elif case == "Wound Lt 1st digit":
      AppenData = "Wound"
    elif case == "Open wound @ ventral abdomen":
      AppenData = "Wound"
    elif case == "โพรงจมูก และแผล":
      AppenData = "Wound"
    elif case == "แผลที่ไหล่ขวา":
      AppenData = "Wound"
    elif case == "แผลที่ขา":
      AppenData = "Wound"
    elif case == "Bone cyst":
      AppenData = "ETC..."
    elif case == "Open wound (Dog bite) at thorax":
      AppenData = "Bite wound"
    elif case == "Abscess/cyst at right scapula":
      AppenData = "Abscess"
    elif case == "Wd (abscess) left cheek":
      AppenData = "Abscess"
    elif case == "Pus from skin (on cefa + griseofulvin from clinic)":
      AppenData = "ETC..."
    elif case == "Pus from liver mass":
      AppenData = "ETC..."
    elif case == "Nasal swab (both)":
      AppenData = "Pleural effusion"
    elif case == "Chronic bite wound abscess":
      AppenData = "Bite wound"
    elif case == "Purulent exudate from abscess (Rt elbow)":
      AppenData = "Abscess"
    elif case == "แผลทะลุช่องท้อง":
      AppenData = "Wound"
    elif case == "Dog bite wound at neck":
      AppenData = "Bite wound"
    elif case == "Left ureter":
      AppenData = "ETC..."
    elif case == "UB mucosa, urolith":
      AppenData = "Ub mucosa"
    elif case == "Calculi (เอาจากนิ่วในกระเพาะปัสสาวะ)":
      AppenData = "ETC..."
    elif case == "Wound @ perineum area":
      AppenData = "Wound"
    elif case == "Wound @ lumbar":
      AppenData = "Wound"
    elif case == "Tracheobronchial lavage":
      AppenData = "ETC..."
    elif case == "Ear wax (both)":
      AppenData = "Ear exudate"
    elif case == "Fluid stifle jt.":
      AppenData = "ETC..."
    elif case == "Infected wound from rupture mass":
      AppenData = "ETC..."
    elif case == "pododermatitis":
      AppenData = "ETC..."
    elif case == "Pus at gingiva":
      AppenData = "ETC..."
    elif case == "Chronic wound (orthopedic device removed)":
      AppenData = "Wound"
    elif case == "Fistula from dermoid cyst":
      AppenData = "ETC..."
    elif case == "Wound @ left FL":
      AppenData = "Wound"
    elif case == "Exudate from chronic wound":
      AppenData = "Wound"
    elif case == "Opened hernia (intestine)":
      AppenData = "ETC..."
    elif case == "Bite wound at left shoulder":
      AppenData = "Bite wound"
    elif case == "Abdomen (bite wound)":
      AppenData = "Bite wound"
    elif case == "Wound @ right hip mass":
      AppenData = "Wound"
    elif case == "Dog bite wound (around cervical area)":
      AppenData = "Bite wound"
    elif case == "Superficial Sprading Pyoderma":
      AppenData = "ETC..."
    elif case == "Pus from papule":
      AppenData = "ETC..."
    elif case == "Urine (both hindlimb paralysis ต้องบีบ UB)":
      AppenData = "Urine"
    elif case == "Unk":
      AppenData = "Unknow"
    elif case == "Crust from Skin":
      AppenData = "ETC..."
    elif case == "BAL":
      AppenData = "Unknow"
    elif case == "Tail":
      AppenData = "Unknow"
    elif case == "bile":
      AppenData = "Bite wound"
    elif case == "Fistula tract":
      AppenData = "ETC..."
    elif case == "Chronic bite wound (recur)":
      AppenData = "ETC..."
    elif case == "Nasal cavity (Rhinoscopy)":
      AppenData = "ETC..."
    elif case == "Screw site (open wound)":
      AppenData = "Wound"
    elif case == "Dead bone in nasal cavity":
      AppenData = "ETC..."
    elif case == "Fistular tract (bite wound)":
      AppenData = "Bite wound"
    elif case == "Open wound @ OD":
      AppenData = "Wound"
    elif case == "Opened wound//Plate fixation 1 month":
      AppenData = "Wound"
    elif case == "Lt.ear cerumen":
      AppenData = "Ear exudate"
    elif case == "Pus from renal pelvis (left)":
      AppenData = "pustule"
    elif case == "Lt.ear cerumen DDx.: Otitis externa":
      AppenData = "Ear exudate"
    elif case == "Wound at back":
      AppenData = "Wound"
    elif case == "Wound @ right FL":
      AppenData = "Wound"
    elif case == "Right hl mass":
      AppenData = "ETC..."
    elif case == "Surgical wound (Cross pin at left distal femur)":
      AppenData = "Surgical Site"
    elif case == "LHL opened fx":
      AppenData = "ETC..."
    elif case == "Exudate wound":
      AppenData = "Wound"
    elif case == "Nasal cavity rhinoscopy":
      AppenData = "Pleural effusion"
    elif case == "Abscess Lt FL":
      AppenData = "Abscess"
    elif case == "Surgical wound":
      AppenData = "Surgical Site"
    elif case == "Bite wound at neck":
      AppenData = "Bite wound"
    elif case == "Abdominal cavity (Bite wound)":
      AppenData = "Bite wound"
    elif case == "Pustule from skin":
      AppenData = "pustule"
    elif case == "Exudate from Rt carpal joint":
      AppenData = "ETC..."
    elif case == "Abscess หลัง":
      AppenData = "Abscess"
    elif case == "Left ear wax":
      AppenData = "Ear exudate"
    elif case == "Surgical site infection":
      AppenData = "Surgical Site"
    elif case == "Wound @ หน้าเท้า of Lt HL":
      AppenData = "Wound"
    elif case == "Abscess @ neck":
      AppenData = "Abscess"
    elif case == "Nasal mass":
      AppenData = "ETC..."
    elif case == "Opened wound (HBC)":
      AppenData = "Wound"
    elif case == "Abdominal cavity (Uroabdomen)":
      AppenData = "Abdominal effusion"
    elif case == "Pustule (Deep pyoderma)/ pododermatitis":
      AppenData = "pustule"
    elif case == "Exudate  (wound)":
      AppenData = "Wound"
    elif case == "Purulent exudate from Rt chest drain":
      AppenData = "pustule"
    elif case == "Pleural fluid":
      AppenData = "Pleural effusion"
    elif case == "Pus from left kidney":
      AppenData = "ETC..."
    elif case == "Exudate from Lt metaphalanges wound":
      AppenData = "Wound"
    elif case == "Chronic wound at rt. scapular area":
      AppenData = "Wound"
    elif case == "Swab from bite wound":
      AppenData = "Bite wound"
    elif case == "หนองจากตุ่มหนองที่ผิวหนังใต้ท้อง":
      AppenData = "pustule"
    elif case == "Opened wound @ Rt.HL(รถชน) >open bone fx":
      AppenData = "Wound"
    elif case == "Aural exudate":
      AppenData = "ETC..."
    elif case == "UB mucosa (ประวัติ Cytic calculi)":
      AppenData = "Ub mucosa"
    elif case == "Abdominal swab":
      AppenData = "Abdominal effusion"
    elif case == "Purulent exudate pododermatitis /acral lick":
      AppenData = "pustule"
    elif case == "Abscess at back":
      AppenData = "Abscess"
    elif case == "Wound with purulent discharge":
      AppenData = "Wound"
    elif case == "Abscess at left inguinal":
      AppenData = "Abscess"
    elif case == "Salivary mucocele":
      AppenData = "ETC..."
    elif case == "Abscess (1st phalange Rt.HL)":
      AppenData = "Abscess"
    elif case == "Stump pyometra":
      AppenData = "ETC..."
    elif case == "Prostatic abscess":
      AppenData = "Abscess"
    elif case == "Skin exudate":
      AppenData = "ETC..."
    elif case == "Bite wound with open fracture":
      AppenData = "Bite wound"
    elif case == "น้ำในช่องท้อง":
      AppenData = "Abdominal effusion"
    elif case == "Exudate from open wound at Lt.FL":
      AppenData = "Wound"
    elif case == "Exudate from wound at tail":
      AppenData = "Wound"
    elif case == "Open wound":
      AppenData = "Wound"
    elif case == "Pus at skin":
      AppenData = "ETC..."
    elif case == "Chronic close fistula wound":
      AppenData = "Wound"
    elif case == "Mammary abscess":
      AppenData = "Abscess"
    elif case == "Skin/sc abscess at Lt.thorax":
      AppenData = "Abscess"
    elif case == "Deep pyoderma หาง":
      AppenData = "ETC..."
    elif case == "nan":
      AppenData = "Unknow"
    elif case == "Ear swab (Chronic wound otitis)":
      AppenData = "Ear exudate"
    elif case == "Exudate from abscess wound":
      AppenData = "Abscess"
    elif case == "Purulent ear wax (Rt.ear)":
      AppenData = "Ear exudate"
    elif case == "Swab from Lt ear canal":
      AppenData = "Ear exudate"
    elif case == "Swab from Rt ear canal":
      AppenData = "Ear exudate"
    elif case == "Nasal swab":
      AppenData = "ETC..."
    elif case == "Deep exudate":
      AppenData = "ETC..."
    elif case == "Deep Exudate":
      AppenData = "ETC..."
    elif case == "Bite wound at right forelimb":
      AppenData = "Bite wound"
    elif case == "แผลที่ rt tarsal jt.":
      AppenData = "Wound"
    elif case == "Contaminated wound":
      AppenData = "Wound"
    elif case == "Wound @ Rt HL":
      AppenData = "Wound"
    elif case == "Wound @ Rt femur":
      AppenData = "Wound"
    elif case == "Abscess bite wound":
      AppenData = "Bite wound"
    elif case == "Wound @ neck":
      AppenData = "Wound"
    elif case == "Swab หนองที่หู":
      AppenData = "Ear exudate"
    elif case == "Urine เคยได้รับ Amoxy-clav และ Marbofloxacin มาก่อนหน้านี้ ปัจจุบันให้Marbofloxacin อยู่":
      AppenData = "Urine"
    elif case == "skin":
      AppenData = "ETC..."
    elif case == "Fluid from mass (serosang+purulent)":
      AppenData = "pustule"
    elif case == "Exudate from the Rt ear":
      AppenData = "Ear exudate"
    elif case == "Abscess @ left lumbar (recurrent)":
      AppenData = "Abscess"
    elif case == "Sc abscess @ suture line":
      AppenData = "Abscess"
    elif case == "Lt ear exudate":
      AppenData = "Ear exudate"
    elif case == "Wound with purulent discharge / mass":
      AppenData = "Wound"
    elif case == "UB mucosa+urine+calculus":
      AppenData = "Ub mucosa"
    elif case == "Chronic open wound at Rt inguinal":
      AppenData = "Wound"
    elif case == "Fistula track @ ventral neck":
      AppenData = "ETC..."
    elif case == "Wound fistula tract at neck":
      AppenData = "Wound"
    elif case == "Wound with purulent discharge (chronic wound)":
      AppenData = "Wound"
    elif case == "Crust from skin; not improve after AMC":
      AppenData = "ETC..."
    elif case == "Open wound at lumbosacral area":
      AppenData = "Wound"
    elif case == "Nasal mucosa":
      AppenData = "Pleural effusion"
    elif case == "Abscess @ dorsal neck":
      AppenData = "Abscess"
    elif case == "Fluid from renal pelvis":
      AppenData = "ETC..."
    elif case == "Wound / Pregnancy patient":
      AppenData = "Wound"
    elif case == "Lt facial abscess":
      AppenData = "Abscess"
    elif case == "Wound":
      AppenData = "Wound"
    elif case == "Wound at lumbar":
      AppenData = "Wound"
    elif case == "Screw from pelvic":
      AppenData = "ETC..."
    elif case == "Abscess at left lumbar area":
      AppenData = "Abscess"
    elif case == "Abscess (Rt. forelimb)":
      AppenData = "Abscess"
    elif case == "Open hernia":
      AppenData = "ETC..."
    elif case == "Bited wound":
      AppenData = "Bite wound"
    elif case == "Swab from cheek":
      AppenData = "ETC..."
    elif case == "Uterine mucosa":
      AppenData = "Ub mucosa"
    sample_Collect.append(AppenData)

  traininput['submitted_sample'] = sample_Collect


  """#Encode Label to value"""

 

  le = LabelEncoder()
  traininput = traininput.apply(lambda col : le.fit_transform(col.astype(str)))

  
  """#make Model

  ##Decision Tree
  """

  

  X_train, X_test, Y_train, Y_test = train_test_split(traininput, trainAnswer, test_size=0.2,random_state=42)

  

  clf = tree.DecisionTreeClassifier()
  clf = clf.fit(X_train, Y_train)


  scores = cross_val_score(clf,traininput,trainAnswer, cv = 10)
  print(medicine)
  print("   Accuracy: %0.2f (+/- %0.2f)" %(scores.mean(), scores.std()*2))
  print("   ",med_volume)

  this_df = pd.DataFrame([[medicine,count,scores,scores.mean(),scores.std()]],columns=['medicine','sample amount','score','mean','std'])
  stat_df = stat_df.append(this_df,ignore_index=True)
  """#save model"""


  filename = '/content/drive/My Drive/ThaiWordcutTest/DecisionTree_Model2/'+medicine+'_decisionTree.sav'

  # pickle.dump(clf, open(filename, 'wb'))
  dot_data = export_graphviz(clf,out_file=None, filled=True, rounded=True, special_characters=False, impurity=True, feature_names=traininput.columns)
  graph = graphviz.Source(dot_data)
  # graph.render(filename=medicine,directory='/content/drive/My Drive/ThaiWordcutTest/DecisionTree_Model2/graph',format='pdf')
  

Amikacin
   Accuracy: 0.95 (+/- 0.04)
    ['15 mg/kg ', '15-30 mg/kg ']
Amoxicillin
   Accuracy: 0.92 (+/- 0.06)
    ['25 mg/kg ', '15-25 mg/kg ']
Clavulanic acid
   Accuracy: 0.92 (+/- 0.06)
    ['25 mg/kg ', '15-25 mg/kg ']


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 6 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Azithromycin
   Accuracy: 0.99 (+/- 0.02)
    ['10 mg/kg ', '5-10 mg/kg ']


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Cefazolin
   Accuracy: 1.00 (+/- 0.01)
    ['20-35 mg/kg ']


...

Gentamicin
   Accuracy: 0.98 (+/- 0.03)
    ['6 mg/kg ', '3-6 mg/kg ']
Marbofloxacin
   Accuracy: 0.94 (+/- 0.04)
    ['11-22 mg/kg ', '5.5-11 mg/kg ', '5.5 mg/kg ']
Nitrofurantoin
   Accuracy: 0.89 (+/- 0.06)
    ['5 mg/kg ']


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Pradofloxacin
   Accuracy: 1.00 (+/- 0.01)
    ['5 mg/kg ']


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 6 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Rifampicin
   Accuracy: 0.98 (+/- 0.04)
    ['5 mg/kg ']
Sulfamethoxazole
   Accuracy: 0.81 (+/- 0.07)
    ['30 mg/kg ', '15-30 mg/kg ']
Vancomycin
   Accuracy: 0.96 (+/- 0.06)
    ['15 mg/kg ', '12-15 mg/kg ']
Enrofloxacin
   Accuracy: 0.95 (+/- 0.05)
    ['10-20 mg/kg ', '5-10 mg/kg ', '15 mg/kg ', '2.5-5 mg/kg ', '10 mg/kg ']
Marbofloxacin
   Accuracy: 0.94 (+/- 0.06)
    ['11-22 mg/kg ', '5.5-11 mg/kg ', '5.5 mg/kg ']


In [ ]:
stat_df

,medicine,sample amount,score,mean,std
0,Amikacin,23,"[0.9180327868852459, 0.9344262295081968, 0.934...",0.950355,0.020697
1,Amoxicillin,379,"[0.9016393442622951, 0.9180327868852459, 0.918...",0.923770,0.028872
2,Clavulanic acid,378,"[0.9016393442622951, 0.9672131147540983, 0.868...",0.922104,0.032135
3,Azithromycin,6,"[0.9836065573770492, 0.9672131147540983, 1.0, ...",0.990082,0.010924
4,Cefazolin,1,"[1.0, 1.0, 0.9836065573770492, 1.0, 1.0, 1.0, ...",0.998361,0.004918
5,Cefixime,1,"[1.0, 1.0, 0.9836065573770492, 1.0, 1.0, 1.0, ...",0.995027,0.010642
6,Cefovecin,209,"[0.8524590163934426, 0.9016393442622951, 0.803...",0.865738,0.037846
7,Ceftriaxone,4,"[0.9836065573770492, 0.9836065573770492, 0.983...",0.985082,0.018923
8,Cephalexin,177,"[0.8524590163934426, 0.9180327868852459, 0.885...",0.855574,0.048602
9,Cephalexin,177,"[0.8852459016393442, 0.8852459016393442, 0.885...",0.853907,0.050240


In [ ]:
stat_df.to_csv('/content/drive/My Drive/ThaiWordcutTest/DecisionTree_Model/Stat_GP.csv', encoding='utf-8')

In [ ]:
len(traininput.columns)

29

In [ ]:
summ = 0
count_med = 0
for index,i in stat_df.iterrows():
  print(i['mean'])
  summ = summ + i['mean']
  print('sum = ' + str(summ))
  count_med = count_med + 1

0.9503551912568305
sum = 0.9503551912568305
0.9237704918032786
sum = 1.8741256830601092
0.9221038251366119
sum = 2.796229508196721
0.9900819672131146
sum = 3.7863114754098355
0.9983606557377049
sum = 4.78467213114754
0.9950273224043716
sum = 5.779699453551912
0.8657377049180328
sum = 6.645437158469945
0.9850819672131147
sum = 7.63051912568306
0.8555737704918032
sum = 8.486092896174863
0.8539071038251367
sum = 9.34
0.7661475409836065
sum = 10.106147540983606
0.9568852459016393
sum = 11.063032786885245
0.9403825136612021
sum = 12.003415300546447
0.9817213114754099
sum = 12.985136612021856
0.9401912568306011
sum = 13.925327868852458
0.888879781420765
sum = 14.814207650273223
0.9983606557377049
sum = 15.812568306010927
0.980191256830601
sum = 16.79275956284153
0.8142622950819673
sum = 17.607021857923495
0.9568579234972677
sum = 18.563879781420763
0.9453551912568307
sum = 19.509234972677593
0.9386612021857923
sum = 20.447896174863384


In [ ]:
count_med

22

In [ ]:
summ

20.447896174863384

In [ ]:
summ/count_med

0.9294498261301538